In [57]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [58]:
df = pd.read_csv('../data/process_v1/process_data_6.csv')
df = df[df['city'] == 0]

In [59]:
df = df[df['landSize'] <= 5000]

In [60]:
df['is_street_house'] = (df['accessibility'] == 2).astype(np.int32)

In [61]:
df['landSize_ratio'] = df['landSize'] / df['acreage']

In [62]:
mean_land_size_df = df.groupby('administrative_genre')['landSize'].mean().reset_index().rename(columns = {'landSize': 'meanLandSize'})

df = df.merge(mean_land_size_df, how='left', on = 'administrative_genre')

df['landSize_ratio_with_administrative_genre'] = df['landSize'] / df['meanLandSize']
df['acreage_ratio_with_meanLandSize'] = df['acreage'] / df['meanLandSize']


del df['meanLandSize']

def partition_for_alley_width(alley_width):
    if math.isnan(alley_width):
        return np.nan
    if alley_width <= 2.5:
        return 1
    if alley_width <= 5:
        return 2
    if alley_width <= 7:
        return 3
    if alley_width <= 8:
        return 4

In [63]:
{'fitThreeCars': 0,
 'fitOneCarAndOneMotorbike': 1,
 'notInTheAlley': 2,
 'parkCar': 3,
 'theBottleNeckPoint': 4,
 'fitTwoCars': 5,
 'narrorRoad': 6}

{'fitThreeCars': 0,
 'fitOneCarAndOneMotorbike': 1,
 'notInTheAlley': 2,
 'parkCar': 3,
 'theBottleNeckPoint': 4,
 'fitTwoCars': 5,
 'narrorRoad': 6}

In [64]:
df.columns.tolist()

['numberOfFloors',
 'numberOfBathRooms',
 'numberOfBedRooms',
 'certificateOfLandUseRight',
 'ward',
 'street',
 'district',
 'city',
 'lat',
 'lon',
 'typeOfRealEstate',
 'frontWidth',
 'endWidth',
 'facade',
 'houseDirection',
 'landSize',
 'price',
 'accessibility',
 'description',
 'time',
 'w',
 'h',
 'num_of_university_in_500m_radius',
 'num_of_fuel_in_500m_radius',
 'num_of_cafe_in_500m_radius',
 'num_of_parking_in_500m_radius',
 'num_of_parking_entrance_in_500m_radius',
 'num_of_fast_food_in_500m_radius',
 'num_of_marketplace_in_500m_radius',
 'num_of_restaurant_in_500m_radius',
 'num_of_hospital_in_500m_radius',
 'num_of_school_in_500m_radius',
 'num_of_kindergarten_in_500m_radius',
 'num_of_townhall - community_centre_in_500m_radius',
 'num_of_police_in_500m_radius',
 'num_of_place_of_worship_in_500m_radius',
 'num_of_bank_in_500m_radius',
 'num_of_atm_in_500m_radius',
 'num_of_university_in_1000m_radius',
 'num_of_fuel_in_1000m_radius',
 'num_of_cafe_in_1000m_radius',
 'num_

In [65]:
s1 = u'ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚÝàáâãèéêìíòóôõùúýĂăĐđĨĩŨũƠơƯưẠạẢảẤấẦầẨẩẪẫẬậẮắẰằẲẳẴẵẶặẸẹẺẻẼẽẾếỀềỂểỄễỆệỈỉỊịỌọỎỏỐốỒồỔổỖỗỘộỚớỜờỞởỠỡỢợỤụỦủỨứỪừỬửỮữỰựỲỳỴỵỶỷỸỹ'
s0 = u'AAAAEEEIIOOOOUUYaaaaeeeiioooouuyAaDdIiUuOoUuAaAaAaAaAaAaAaAaAaAaAaAaEeEeEeEeEeEeEeEeIiIiOoOoOoOoOoOoOoOoOoOoOoOoUuUuUuUuUuUuUuYyYyYyYy'
def remove_accents(input_str):
	s = ''
	for c in input_str:
		if c in s1:
			s += s0[s1.index(c)]
		else:
			s += c

	s = "".join([c for c in list(s) if c == '_' or c.isalnum()])
	return s

full_cols = df.columns.tolist()
format_full_cols = [remove_accents(col) for col in full_cols]

df = df.rename(columns = dict(zip(full_cols, format_full_cols)))

In [66]:
cat_cols = ['is_street_house', 'ward', 'street', 'district', 'typeOfRealEstate', 'houseDirection', 'accessibility', 'certificateOfLandUseRight'] + [c for c in df.columns.tolist() if ('_district' in c or '_ward' in c or '_street' in c) and 'distance' not in c and 'num' not in c]
target_cols = ['target']
remove_cols = ['price', 'description', 'city', 'endWidth'] + [c for c in df.columns if 'distance_hcm' in c]
time_cols = ['time']
num_cols = [c for c in df.columns.tolist() if c not in (cat_cols + target_cols + remove_cols + time_cols)]

cat_cols = list(set(cat_cols))
target_cols = list(set(target_cols))
remove_cols = list(set(remove_cols))
num_cols = list(set(num_cols))

print(len(cat_cols), len(target_cols), len(remove_cols), len(num_cols))

35 1 53 148


In [67]:
cat_cols

['nearest_2_district',
 'nearest_2_ward',
 'accessibility',
 'nearest_3_district',
 'nearest_4_ward',
 'nearest_7_street',
 'nearest_0_district',
 'houseDirection',
 'nearest_8_district',
 'nearest_0_street',
 'nearest_8_street',
 'nearest_6_ward',
 'nearest_3_street',
 'nearest_7_ward',
 'nearest_8_ward',
 'nearest_5_district',
 'nearest_4_district',
 'nearest_6_street',
 'nearest_1_ward',
 'is_street_house',
 'nearest_2_street',
 'nearest_3_ward',
 'ward',
 'typeOfRealEstate',
 'district',
 'certificateOfLandUseRight',
 'nearest_1_district',
 'nearest_1_street',
 'nearest_0_ward',
 'nearest_4_street',
 'nearest_7_district',
 'nearest_5_ward',
 'nearest_5_street',
 'street',
 'nearest_6_district']

In [68]:
num_cols

['num_of_kindergarten_in_2000m_radius',
 'distance_hn_Congvien239',
 'distance_hn_DuongMacThiBuoi',
 'distance_hn_TranHungDao',
 'nearest_7_lat',
 'nearest_5_lat',
 'num_of_parking_entrance_in_1000m_radius',
 'distance_hn_SCVivoCityShoppingCenter',
 'num_of_police_in_2000m_radius',
 'distance_hn_SaigonGarden',
 'distance_hn_VincomDongKhoi',
 'num_of_parking_in_2000m_radius',
 'distance_hn_DuongNguyenTatThanh',
 'num_of_school_in_1000m_radius',
 'nearest_1_lat',
 'distance_nearest_6',
 'num_of_hospital_in_2000m_radius',
 'landSize_ratio',
 'distance_hn_CongvienTaoDan',
 'numberOfBedRooms',
 'distance_hn_NguyenVanLinh',
 'num_of_kindergarten_in_500m_radius',
 'district_lat',
 'num_of_cafe_in_2000m_radius',
 'distance_nearest_2',
 'distance_hn_DuongNguyenHuequan1TPHCM',
 'num_of_police_in_1000m_radius',
 'distance_hn_DaoTri',
 'nearest_6_lon',
 'distance_hn_TonThatThuyet',
 'nearest_3_lat',
 'distance_nearest_1',
 'num_of_university_in_1000m_radius',
 'administrative_genre',
 'num_of_bank

In [69]:
print(len(num_cols))

148


In [70]:
df.columns.tolist()

['numberOfFloors',
 'numberOfBathRooms',
 'numberOfBedRooms',
 'certificateOfLandUseRight',
 'ward',
 'street',
 'district',
 'city',
 'lat',
 'lon',
 'typeOfRealEstate',
 'frontWidth',
 'endWidth',
 'facade',
 'houseDirection',
 'landSize',
 'price',
 'accessibility',
 'description',
 'time',
 'w',
 'h',
 'num_of_university_in_500m_radius',
 'num_of_fuel_in_500m_radius',
 'num_of_cafe_in_500m_radius',
 'num_of_parking_in_500m_radius',
 'num_of_parking_entrance_in_500m_radius',
 'num_of_fast_food_in_500m_radius',
 'num_of_marketplace_in_500m_radius',
 'num_of_restaurant_in_500m_radius',
 'num_of_hospital_in_500m_radius',
 'num_of_school_in_500m_radius',
 'num_of_kindergarten_in_500m_radius',
 'num_of_townhallcommunity_centre_in_500m_radius',
 'num_of_police_in_500m_radius',
 'num_of_place_of_worship_in_500m_radius',
 'num_of_bank_in_500m_radius',
 'num_of_atm_in_500m_radius',
 'num_of_university_in_1000m_radius',
 'num_of_fuel_in_1000m_radius',
 'num_of_cafe_in_1000m_radius',
 'num_of_

In [71]:
df[cat_cols] = df[cat_cols].fillna(100)
df[cat_cols] = df[cat_cols].astype(np.int32)

In [72]:
df[num_cols]

,num_of_kindergarten_in_2000m_radius,distance_hn_Congvien239,distance_hn_DuongMacThiBuoi,distance_hn_TranHungDao,nearest_7_lat,nearest_5_lat,num_of_parking_entrance_in_1000m_radius,distance_hn_SCVivoCityShoppingCenter,num_of_police_in_2000m_radius,distance_hn_SaigonGarden,...,frontWidth,num_of_school_in_2000m_radius,distance_hn_NguyenVanLuong,population,num_of_restaurant_in_1000m_radius,district_lon,num_of_place_of_worship_in_2000m_radius,distance_hn_LeHongPhong,distance_hn_Congvien304,distance_hn_KinhDuongVuong
0,2,9.348445,9.240799,9.457042,NaN,NaN,0,9.613846,0,9.618949,...,0.0,19,9.895673,13.170525,1,106.625025,20,9.509579,9.232693,9.859252
1,2,8.648254,8.697570,8.801644,10.726797,10.727941,0,6.460449,4,9.538110,...,0.0,26,9.758678,12.690411,30,106.645937,3,8.749448,8.802010,9.310765
2,4,8.246297,8.225249,8.580951,10.747520,10.741552,0,7.531876,5,9.446127,...,4.0,31,9.714820,12.690411,6,106.645937,30,8.564476,8.413145,9.313806
3,9,8.662036,8.769858,8.425273,10.800510,10.797977,0,9.301450,3,8.209004,...,0.0,54,9.046271,13.061232,8,106.612072,45,8.529008,8.617991,8.948865
4,13,7.838244,7.661843,8.175547,10.789980,10.793641,0,8.936273,5,9.040266,...,4.0,66,9.482063,12.231643,100,106.674663,83,8.352073,7.230999,9.234083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48888,0,9.677173,9.707239,9.593890,NaN,NaN,0,9.958589,0,8.964934,...,14.0,7,8.995150,13.000130,0,106.429011,2,9.620117,9.650468,9.559535
48889,3,9.531314,9.543555,9.470783,10.874777,10.865114,0,9.859584,0,8.946028,...,4.0,10,9.227670,13.161921,2,106.578293,6,9.511700,9.479769,9.582549
48890,2,9.574764,9.473543,9.691810,10.856760,10.848256,0,9.689969,7,9.903854,...,5.0,17,10.122087,12.891692,0,106.594480,9,9.727426,9.496038,10.044954
48891,13,7.833259,8.264963,7.428294,10.754380,10.754168,0,8.552219,5,9.065112,...,0.0,83,9.411029,14.442256,37,106.668792,58,7.022286,8.225567,8.835211


In [73]:
df = df[cat_cols + num_cols + time_cols + target_cols]

In [74]:
df = df[df[target_cols[0]] != np.inf]
df = df[df[target_cols[0]] != -np.inf]

In [75]:
df.shape

(48893, 185)

In [76]:
df = df[df[target_cols[0]] >= 10]
df = df[df[target_cols[0]] < 100 * 10]
# df = pd.concat([df[df['w'] < 50], df[df['w'].isnull()]])
# df = pd.concat([df[df['h'] < 50], df[df['w'].isnull()]])

In [77]:
for col in num_cols:
    df[col] = df[col].fillna(df[col].mean())

In [78]:
for c in df.columns.tolist():
    if df[c].isnull().sum():
        print(c)
        del df[c]

In [79]:
gmm_config = {
    'num_of_restaurant_in_2000m_radius': 2,
    'num_of_restaurant_in_1000m_radius': 2,
    'num_of_bank_in_2000m_radius': 3,
    'num_of_marketplace_in_2000m_radius': 2,
    'num_of_cafe_in_1000m_radius': 2
}

In [85]:
from sklearn.mixture import GaussianMixture
from joblib import dump, load

In [81]:
gmm_dict = {}

In [82]:
for col in gmm_config.keys():
    gmm = GaussianMixture(n_components=gmm_config[col], random_state=42)
    gmm.fit(df[col].values.reshape(-1, 1))
    df[f'gmm_{gmm_config[col]}_component_{col}'] = gmm.predict(df[col].values.reshape(-1, 1))

    gmm_dict[col] = gmm

/tmp/ipykernel_364518/1287373673.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'gmm_{gmm_config[col]}_component_{col}'] = gmm.predict(df[col].values.reshape(-1, 1))
/tmp/ipykernel_364518/1287373673.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'gmm_{gmm_config[col]}_component_{col}'] = gmm.predict(df[col].values.reshape(-1, 1))
/tmp/ipykernel_364518/1287373673.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.

In [83]:
df[f'gmm_{gmm_config[col]}_component_{col}']

0        0
1        1
2        0
3        0
4        1
        ..
48888    0
48889    0
48890    0
48891    0
48892    1
Name: gmm_2_component_num_of_cafe_in_1000m_radius, Length: 44149, dtype: int64

In [87]:
for col in gmm_config.keys():
    dump(gmm_dict[col], f"./hcm_gmm_{col}.joblib")

In [ ]:
cat_cols = ['is_street_house', 'ward', 'street', 'district', 'typeOfRealEstate', 'houseDirection', 'accessibility', 'certificateOfLandUseRight'] + [c for c in df.columns.tolist() if ('_district' in c or '_ward' in c or '_street' in c) and 'distance' not in c and 'num' not in c]
cat_cols += [c for c in df.columns if 'gmm' in c]
cat_cols = list(set(cat_cols))
target_cols = ['target']
remove_cols = ['price', 'description', 'city', 'endWidth'] + [c for c in df.columns if 'distance_hcm' in c]
time_cols = ['time']
num_cols = [c for c in df.columns.tolist() if c not in (cat_cols + target_cols + remove_cols + time_cols)]

cat_cols = list(set(cat_cols))
target_cols = list(set(target_cols))
remove_cols = list(set(remove_cols))
num_cols = list(set(num_cols))

print(len(cat_cols), len(target_cols), len(remove_cols), len(num_cols))

In [ ]:
cat_cols

In [ ]:
num_cols

In [ ]:
df = df.reset_index(drop = True)
df.to_csv('../data/process_v1/process_data_8_hcm.csv', index = False)